In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-12 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-12 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-12 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-12 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-12 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[128,128],[256,256]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [8]:
hidden_dim_sizes

[[128, 128], [256, 256]]

In [9]:
learning_rates

[0.001, 0.0001, 0.0005]

We use the `ffn_hyperparameter_search` function which loops through the different hidden dimensions, dropout rates and learning rates to find the best model for the validation set. We evaluate the model on several seeds and average the performance over the seeds.

In [10]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data_client,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=False,
    patience=patience,    
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_best_model.csv


In [11]:
ffn_current

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,model_id
0,0.549651,0.691450,0.626895,"[0.7819181429444105, 0.5555555555555557, 0.543...",0.619121,"[0.794044665012407, 0.5678233438485805, 0.4954...",0.638352,"[0.7701564380264742, 0.5438066465256798, 0.601...",0.526311,0.703532,...,"(128, 128)",0.5,0.0010,0,focal,2,False,None,64,0
0,0.594199,0.692937,0.614575,"[0.7898763978811066, 0.5261437908496732, 0.527...",0.618733,"[0.7730414746543779, 0.5729537366548043, 0.510...",0.613438,"[0.8074608904933814, 0.486404833836858, 0.5464...",0.556140,0.710967,...,"(128, 128)",0.5,0.0010,1,focal,2,False,None,64,0
0,0.568504,0.710781,0.635073,"[0.8083086053412464, 0.5437201907790142, 0.553...",0.636703,"[0.797423887587822, 0.5738255033557047, 0.5388...",0.634806,"[0.8194945848375451, 0.5166163141993958, 0.568...",0.527960,0.719331,...,"(128, 128)",0.5,0.0010,12,focal,2,False,None,64,0
0,0.665234,0.689219,0.613046,"[0.7880597014925372, 0.5279999999999999, 0.523...",0.611990,"[0.7819905213270142, 0.5612244897959183, 0.492...",0.616697,"[0.7942238267148014, 0.4984894259818731, 0.557...",0.617006,0.716543,...,"(128, 128)",0.5,0.0010,123,focal,2,False,None,64,0
0,0.604502,0.701859,0.623319,"[0.7964912280701756, 0.536741214057508, 0.5367...",0.633264,"[0.7747440273037542, 0.5694915254237288, 0.555...",0.615391,"[0.8194945848375451, 0.5075528700906344, 0.519...",0.533370,0.726766,...,"(128, 128)",0.5,0.0010,1234,focal,2,False,None,64,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.602591,0.686989,0.616372,"[0.7854984894259818, 0.5229357798165138, 0.540...",0.612816,"[0.7888349514563107, 0.5294117647058824, 0.520...",0.620549,"[0.7821901323706378, 0.5166163141993958, 0.562...",0.554826,0.712825,...,"(256, 256)",0.1,0.0005,0,focal,2,False,None,64,17
0,0.629131,0.707807,0.621480,"[0.8038658328595794, 0.5130890052356021, 0.547...",0.643097,"[0.7618534482758621, 0.6074380165289256, 0.56]",0.610137,"[0.8507821901323707, 0.44410876132930516, 0.53...",0.560590,0.729554,...,"(256, 256)",0.1,0.0005,1,focal,2,False,None,64,17
0,0.600558,0.695167,0.624620,"[0.7874481941977501, 0.5302325581395348, 0.556...",0.630633,"[0.7750582750582751, 0.5445859872611465, 0.572...",0.619280,"[0.8002406738868832, 0.5166163141993958, 0.540...",0.553091,0.716543,...,"(256, 256)",0.1,0.0005,12,focal,2,False,None,64,17
0,0.552720,0.689219,0.619736,"[0.7851941747572816, 0.5390749601275916, 0.534...",0.613772,"[0.791921664626683, 0.5709459459459459, 0.4784...",0.631904,"[0.7785800240673887, 0.5105740181268882, 0.606...",0.526706,0.703532,...,"(256, 256)",0.1,0.0005,123,focal,2,False,None,64,17


In [12]:
ffn_current.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_59024/3305696319.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_current.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


loss  accuracy        f1  \
hidden_dim dropout_rate learning_rate                                 
(128, 128) 0.1          0.0001         0.574906  0.681784  0.601424   
                        0.0005         0.594163  0.691004  0.616426   
                        0.0010         0.610184  0.698290  0.620500   
           0.2          0.0001         0.561039  0.688030  0.607769   
                        0.0005         0.567306  0.690260  0.618901   
                        0.0010         0.604894  0.690706  0.612315   
           0.5          0.0001         0.558901  0.683420  0.603879   
                        0.0005         0.629294  0.702156  0.624711   
                        0.0010         0.596418  0.697249  0.622581   
(256, 256) 0.1          0.0001         0.555807  0.695762  0.621950   
                        0.0005         0.587081  0.695167  0.620529   
                        0.0010         0.604229  0.693680  0.617978   
           0.2          0.0001         0.558192  0.690409  0.615866   
                        0.0005         0.651493  0.696208  0.617660   
                        0.0010         0.679231  0.698587  0.619034   
           0.5          0.0001         0.558001  0.688625  0.612047   
                        0.0005         0.594275  0.688625  0.612354   
                        0.0010         0.602223  0.699033  0.621951   

                                       precision    recall  valid_loss  \
hidden_dim dropout_rate learning_rate                                    
(128, 128) 0.1          0.0001          0.600488  0.606050    0.529534   
                        0.0005          0.618782  0.616015    0.551671   
                        0.0010          0.626817  0.618954    0.552827   
           0.2          0.0001          0.609206  0.610761    0.524637   
                        0.0005          0.616278  0.624341    0.536522   
                        0.0010          0.616150  0.611454    0.549207   
           0.5          0.0001          0.602860  0.611081    0.533563   
                        0.0005          0.629072  0.621626    0.526842   
                        0.0010          0.623962  0.623737    0.552157   
(256, 256) 0.1          0.0001          0.620353  0.625191    0.520478   
                        0.0005          0.624526  0.620311    0.542819   
                        0.0010          0.623131  0.618053    0.556276   
           0.2          0.0001          0.613134  0.620152    0.520636   
                        0.0005          0.628040  0.610964    0.581150   
                        0.0010          0.626726  0.615398    0.613856   
           0.5          0.0001          0.611732  0.615569    0.519621   
                        0.0005          0.614581  0.612266    0.549986   
                        0.0010          0.627489  0.619378    0.546661   

                                       valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                             
(128, 128) 0.1          0.0001               0.698141  0.600274   
                        0.0005               0.708364  0.620755   
                        0.0010               0.716171  0.623129   
           0.2          0.0001               0.703903  0.604358   
                        0.0005               0.706134  0.620940   
                        0.0010               0.714312  0.622824   
           0.5          0.0001               0.701487  0.604370   
                        0.0005               0.715242  0.620903   
                        0.0010               0.715428  0.624854   
(256, 256) 0.1          0.0001               0.702230  0.608826   
                        0.0005               0.716543  0.625534   
                        0.0010               0.712268  0.621847   
           0.2          0.0001               0.702416  0.611168   
                        0.0005               0.718773  0.626042   
                        0.0010               0.716357  0.623066   
  

In [13]:
best_ffn_current

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.798638,0.690706,0.604869,"[0.7867902665121669, 0.5293185419968304, 0.498...",0.622886,"[0.758659217877095, 0.5566666666666666, 0.5533...",0.591724,"[0.8170878459687124, 0.5045317220543807, 0.453...",0.702130,0.722119,...,"[0.8372093023255814, 0.5148148148148148, 0.525...","(256, 256)",0.2,0.0005,0,focal,2,False,None,64
0,0.601338,0.712268,0.627379,"[0.8061573546180161, 0.5310344827586206, 0.544...",0.648383,"[0.7659804983748646, 0.6184738955823293, 0.560...",0.615365,"[0.8507821901323707, 0.4652567975830816, 0.530...",0.548570,0.723048,...,"[0.8575581395348837, 0.4740740740740741, 0.508...","(256, 256)",0.2,0.0005,1,focal,2,False,None,64
0,0.614136,0.685502,0.619755,"[0.7782529016493587, 0.5435435435435436, 0.537...",0.613475,"[0.7903225806451613, 0.5402985074626866, 0.509...",0.627227,"[0.766546329723225, 0.5468277945619335, 0.5683...",0.544830,0.710037,...,"[0.7877906976744186, 0.5666666666666667, 0.584...","(256, 256)",0.2,0.0005,12,focal,2,False,None,64
0,0.655915,0.691450,0.614915,"[0.7875220718069453, 0.5220125786163523, 0.535...",0.622442,"[0.7707373271889401, 0.5442622950819672, 0.552...",0.608563,"[0.8050541516245487, 0.5015105740181269, 0.519...",0.578037,0.716543,...,"[0.8212209302325582, 0.5370370370370371, 0.516...","(256, 256)",0.2,0.0005,123,focal,2,False,None,64
0,0.587439,0.701115,0.621380,"[0.7978910369068541, 0.5329153605015674, 0.533...",0.633015,"[0.7773972602739726, 0.5537459283387622, 0.567...",0.611941,"[0.8194945848375451, 0.513595166163142, 0.5027...",0.532184,0.722119,...,"[0.8313953488372093, 0.5444444444444444, 0.491...","(256, 256)",0.2,0.0005,1234,focal,2,False,None,64


In [14]:
best_ffn_current["f1"].mean()

0.6176595172553891

In [15]:
best_ffn_current["precision"].mean()

0.6280401259984101

In [16]:
best_ffn_current["recall"].mean()

0.6109638432313564

In [17]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.79132273, 0.5317649 , 0.52989092])

In [18]:
np.stack(best_ffn_current["precision_scores"]).mean(axis=0)

array([0.77261938, 0.56268946, 0.54881154])

In [19]:
np.stack(best_ffn_current["recall_scores"]).mean(axis=0)

array([0.81179302, 0.50634441, 0.5147541 ])

## KFold

We can repeat this but use K-Fold evaluation instead - by default, we have $K=5$ folds.

In [20]:
ffn_current_kfold, best_ffn_current_kfold, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data_client,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold_best_model.csv


In [21]:
ffn_current_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size,model_id
0,None,0.679554,0.594416,"[0.7839315426669835, 0.5273269294468367, 0.471...",0.593137,"[0.7857993805098881, 0.5891931902294597, 0.404...",0.608652,"[0.7820725634337207, 0.47721822541966424, 0.56...",None,0.680856,...,"(128, 128)",0.5,0.0010,0,focal,2,True,5,64,0
0,None,0.678216,0.592342,"[0.7822053051028904, 0.5290529695024078, 0.465...",0.588954,"[0.7847255369928401, 0.5694540428472702, 0.412...",0.602743,"[0.779701209390562, 0.4940047961630695, 0.5345...",None,0.680405,...,"(128, 128)",0.5,0.0010,1,focal,2,True,5,64,0
0,None,0.679851,0.594282,"[0.7829797321322745, 0.5301681503461919, 0.469...",0.593202,"[0.7827014218009478, 0.589010989010989, 0.4078...",0.606281,"[0.7832582404553, 0.48201438848920863, 0.55357...",None,0.682207,...,"(128, 128)",0.5,0.0010,12,focal,2,True,5,64,0
0,None,0.681338,0.592877,"[0.785018198896325, 0.5217391304347827, 0.471875]",0.593758,"[0.7774418604651163, 0.5843866171003718, 0.419...",0.601084,"[0.7927436566279346, 0.4712230215827338, 0.539...",None,0.686824,...,"(128, 128)",0.5,0.0010,123,focal,2,True,5,64,0
0,None,0.684907,0.597530,"[0.7882422104644325, 0.5267047363117232, 0.477...",0.599105,"[0.7817164179104478, 0.5989304812834224, 0.416...",0.608142,"[0.7948778752667773, 0.47002398081534774, 0.55...",None,0.686036,...,"(128, 128)",0.5,0.0010,1234,focal,2,True,5,64,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.683717,0.593502,"[0.7881266799111838, 0.518, 0.4743792921288959]",0.595564,"[0.7769585253456222, 0.5833333333333334, 0.426...",0.599991,"[0.7996205833530946, 0.4658273381294964, 0.534...",None,0.684572,...,"(256, 256)",0.1,0.0005,0,focal,2,True,5,64,17
0,None,0.681636,0.597157,"[0.7838836571701038, 0.5352904434728295, 0.472...",0.592532,"[0.788111217641419, 0.5586701434159062, 0.4308...",0.605370,"[0.779701209390562, 0.5137889688249401, 0.5226...",None,0.684910,...,"(256, 256)",0.1,0.0005,1,focal,2,True,5,64,17
0,None,0.680595,0.591522,"[0.7858486130700516, 0.5178273908697101, 0.470...",0.592348,"[0.7790724772780238, 0.5828957239309828, 0.415...",0.600873,"[0.7927436566279346, 0.4658273381294964, 0.544...",None,0.687725,...,"(256, 256)",0.1,0.0005,12,focal,2,True,5,64,17
0,None,0.676729,0.591386,"[0.7811792733770101, 0.5245379222434672, 0.468...",0.587196,"[0.7848252752513164, 0.5598639455782313, 0.416...",0.601832,"[0.7775669907517192, 0.4934052757793765, 0.534...",None,0.686712,...,"(256, 256)",0.1,0.0005,123,focal,2,True,5,64,17


In [22]:
ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_59024/3418571787.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
(128, 128) 0.1          0.0001         0.677651  0.590137   0.586863   
                        0.0005         0.678900  0.592796   0.590060   
                        0.0010         0.675063  0.591524   0.588780   
           0.2          0.0001         0.677323  0.589975   0.586790   
                        0.0005         0.678067  0.592232   0.589334   
                        0.0010         0.675242  0.591372   0.588533   
           0.5          0.0001         0.681368  0.593215   0.591568   
                        0.0005         0.677204  0.592454   0.589087   
                        0.0010         0.680773  0.594289   0.593631   
(256, 256) 0.1          0.0001         0.679375  0.592625   0.589402   
                        0.0005         0.679375  0.593361   0.590625   
                        0.0010         0.679435  0.595201   0.591871   
           0.2          0.0001         0.680387  0.593694   0.590658   
                        0.0005         0.679048  0.594052   0.591344   
                        0.0010         0.679673  0.594855   0.591909   
           0.5          0.0001         0.680654  0.594252   0.591589   
                        0.0005         0.681249  0.596677   0.593432   
                        0.0010         0.679316  0.594749   0.592242   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
(128, 128) 0.1          0.0001         0.598913        0.682432  0.594625   
                        0.0005         0.602531        0.681419  0.594574   
                        0.0010         0.607196        0.675833  0.590729   
           0.2          0.0001         0.598785        0.682140  0.594650   
                        0.0005         0.602979        0.682072  0.595385   
                        0.0010         0.605359        0.678221  0.592705   
           0.5          0.0001         0.600476        0.685113  0.597543   
                        0.0005         0.604549        0.680698  0.595554   
                        0.0010         0.605381        0.683266  0.597289   
(256, 256) 0.1          0.0001         0.600437        0.685811  0.599698   
                        0.0005         0.602834        0.684752  0.599521   
                        0.0010         0.606563        0.681441  0.596480   
           0.2          0.0001         0.602070        0.685811  0.599175   
                        0.0005         0.605490        0.683356  0.598442   
                        0.0010         0.605242        0.681982  0.596553   
           0.5          0.0001         0.602625        0.687320  0.601652   
                        0.0005         0.607024        0.685833  0.601362   
                        0.0010         0.607178        0.683041  0.599364   

                                       valid_precision  valid_recall   seed  \
hidden_dim dropout_rate learning_rate                                         
(128, 128) 0.1          0.0001                0.591020      0.603609  274.0   
                        0.0005                0.591630      0.604693  274.0   
                        0.0010                0.587592      0.607079  274.0   
           0.2          0.0001                0.591293      0.603763  274.0   
                        0.0005                0.592538      0.606522  274.0   
                        0.0010                0.589936      0.606792  274.0   
           0.5          0.0001                0.595164      0.605714  274.0   
                        0.0005                0.591685      0.608889  274.0   
                        0.0010                0.595758      0.609610  274.0   
(256, 256) 0.1          0.0001                0.596390      0.607854  274.0   
                        0.0005                0.596389      0.609464  274.0   
                        0.0010

In [23]:
best_ffn_current_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.678662,0.590932,"[0.7821770476303036, 0.5281918340894362, 0.462...",0.589455,"[0.779688972667295, 0.57475317348378, 0.413922...",0.599033,"[0.7846810528811952, 0.48860911270983215, 0.52...",None,0.687950,...,"[0.7903110475509475, 0.5051825146462371, 0.531...","(256, 256)",0.5,0.0001,0,focal,2,True,5,64
0,None,0.679405,0.593399,"[0.7827430473021155, 0.5312101910828025, 0.466...",0.589915,"[0.7846080533714558, 0.5665760869565217, 0.418...",0.602359,"[0.7808868864121413, 0.5, 0.5261904761904762]",None,0.685811,...,"[0.782445477297104, 0.5114916629112213, 0.5417...","(256, 256)",0.5,0.0001,1,focal,2,True,5,64
0,None,0.681784,0.593866,"[0.7856635911994323, 0.530298172491183, 0.4656...",0.591723,"[0.7838092990323342, 0.5699517574086836, 0.421...",0.601189,"[0.7875266777329856, 0.4958033573141487, 0.520...",None,0.686486,...,"[0.7861994994637111, 0.5056331680937359, 0.539...","(256, 256)",0.5,0.0001,12,focal,2,True,5,64
0,None,0.681190,0.596311,"[0.7837548984681155, 0.5306905370843991, 0.474...",0.592844,"[0.7849666983824929, 0.5684931506849316, 0.425...",0.605685,"[0.7825468342423524, 0.49760191846522783, 0.53...",None,0.686937,...,"[0.7860207365033965, 0.5132942767012167, 0.528...","(256, 256)",0.5,0.0001,123,focal,2,True,5,64
0,None,0.682230,0.596753,"[0.7838350319981038, 0.5353373840741925, 0.471...",0.594008,"[0.7834636342099028, 0.5736806031528444, 0.424...",0.604859,"[0.7842067820725634, 0.5017985611510791, 0.528...",None,0.689414,...,"[0.7885234179478012, 0.5150968904912122, 0.532...","(256, 256)",0.5,0.0001,1234,focal,2,True,5,64


In [24]:
best_ffn_current_kfold["f1"].mean()

0.5942523670840995

In [25]:
best_ffn_current_kfold["precision"].mean()

0.5915889248981374

In [26]:
best_ffn_current_kfold["recall"].mean()

0.6026250312463874

In [27]:
np.stack(best_ffn_current_kfold["f1_scores"]).mean(axis=0)

array([0.78363472, 0.53114562, 0.46797675])

In [28]:
np.stack(best_ffn_current_kfold["precision_scores"]).mean(axis=0)

array([0.78330733, 0.57069095, 0.42076849])

In [29]:
np.stack(best_ffn_current_kfold["recall_scores"]).mean(axis=0)

array([0.78396965, 0.49676259, 0.52714286])